In [ ]:
# run once only
import os
os.chdir("../")
print(os.getcwd())

In [1]:
from avinoise import config, preprocess

params = config.params()
params.summary()


****************************Param summary****************************
	Name			|	Value
---------------------------------------------------------------------
	modif			|	2023-03-20 09:55:47.835116
	sr			|	16384
	mono			|	True
	audio_len		|	5
	n_mels			|	32
	n_fft			|	2048
	hop_length		|	256
	win_length		|	2048
	window			|	hann
	center			|	True
	pad_mode		|	reflect
	normalize		|	True
	cutoff			|	0.01
	data_root		|	./data
	augmentation_source	|	.\raw_data\augmentation_source
	raw_clean_path		|	.\raw_data\clean
	raw_contaminated_path	|	.\raw_data\contaminated
	model_type		|	2
	classes			|	['clean', 'contaminated']
	n_batches		|	64
	n_epochs		|	10
	metrics			|	['Precision', 'Recall', 'CategoricalAccuracy']
	tensorboard		|	True
	verbose			|	False
	n_frames		|	321
*********************************************************************



In [ ]:
preprocess.trainTestSplit([params.raw_clean_path, params.raw_contaminated_path])
preprocess.sliceSamples(params.data_root)

In [ ]:
preprocess.evenDataset(os.path.join(params.data_root, "train"))
preprocess.evenDataset(os.path.join(params.data_root, "test"))

In [3]:
from augmolino import augmentation, augmenter

target_folders = [os.path.join(params.data_root, "train", params.classes[0]),
                  os.path.join(params.data_root, "train", params.classes[1]),
                  os.path.join(params.data_root, "test", params.classes[0]),
                  os.path.join(params.data_root, "test", params.classes[1])]

augs = [augmentation.timeStretch(rate=0.75),
        augmentation.timeStretch(rate=1.5),
        augmentation.pitchShift(semitones=-1),
        augmentation.pitchShift(semitones=1),
        augmentation.pitchShift(semitones=2),
        augmentation.offsetAudio(s=1),
        augmentation.offsetAudio(s=-1)]

augmenter_instance = augmenter.augmenter(augmentations=augs)
for target in target_folders:
        augmenter_instance.execute(source=target, dest="auto")